# Introduction to Naive Bayes Models

### Stephen Elston
### Data Science 350

This notebook introduces you to naive Bayes models. Naive Bayes models are a surprisingly useful and effective simplification of the general Bayesian models. Naive Bayes models make the naive assumption of independence of the features.

Some properties of naive Bayes models are:

- Do not require a prior.
- Computational complexity is linear in number of parameter/features.
- Requires minimal data to produce models that generalizes well.
- Have a simple and inherent regularization.

Naive Bayes models are widely used including for:

- Document classification
- SPAM detection
- Image classification


## Overview of Naive Bayes Models

Recall Bayes theorem:

$$P(A|B) = P(B|A) \frac{P(A)}{P(B)}$$

Using the chain rule of conditional probabilities, we can create write the joint distribution for the probability of class $C_k$ as: 

$$
p(C_k, x_1, x_2, x_3, \ldots, x_n) = p(x_1, x_2, x_3, \ldots, x_n, C_k) \\
=  p(x_1 | x_2, x_3, \ldots, x_n, C_k)  p(x_2, x_3, \ldots, x_n, C_k) \\
= p(x_1 | x_2, x_3, \ldots, x_n, C_k)  p(x_2 | x_3, \ldots, x_n, C_k) p(x_3, \ldots, x_n, C_k) \\
\cdots \cdots \\\
=  p(x_1 | x_2, x_3, \ldots, x_n, C_k)  p(x_2 | x_3, \ldots, x_n, C_k) \dots p(C_k)
$$

Let the features, $\{ x_1, x_2, x_3, \ldots, x_n \}$, be independent, the above can then be written as:

$$p(x_i | x_{i + 1}, x_{i + 2}, \ldots, x_n, C_k) = p(x_i | C_k)$$

This simpification allows us to write the probability of the class $C_k$ as the conditional distribution:

$$p(C_k | x_1, x_2, x_3, \ldots, x_n) \propto p(C_k) \prod^N_{j = 1} p(x_j|C_k)$$

Given a number of classes, we can find the mostly likely class $\hat{y}$ as:

$$\hat{y} = argmax_k \Big[ \prod^N_{j = 1} p(x_j|C_k) \Big]$$

Notice that the above formulation uses only the emperical probabilities of the features conditioned on the class. Futher no prior distirbution is required. 

### Pitfalls in naive Bayes models

There are some well know pitfalls with known solutions, including:

- Multiplication of small probabilities leads to floating point underflow. This problem is corrected by computing with the log probabilities, $ln(p)$.
- If there are no samples/data then $p(x_j|C_k) = 0$, leading the product of probabilities to be 0. A Laplace smoother is used to ensure that all $p(x_.j|C_k) > 0$
- Colinear features do not exhibit independence. Ideally, such features should be removed from the the data set to prevent problems with the  model.
- Regularization is generally a minor issue with naive Bayes models, as uninformative feature tend to a uniform distribution which does not affect the outcome.

### Types of naive Bayes models

Now that we have looked into the basics of a naive Bayes models, lets look at some specific formulations. It is important to keep in mind that a specific naive Bayes model is required for each class of problem.  

The Multinomial naive Bayes classifier is a widely used form of the model. The Multinomial classifier finds the mostly likely class from multiple possibilities. To prevent numerical underflow we write this classifier taking the logarithms of both sides of the equation as follows:

$$log \Big( p(C_k | x) \Big) \propto\ log \Big( p(C_k) \prod^N_{j = 1} p_{kj}(x_i) \Big)\\
= log \Big( p(C_k) \Big) + log \Big( \sum^N_{j = 1} p_{kj}(x_i) \Big)$$

The most likely class $\hat{y}$ is then:

$$\hat{y} = argmax_k \Big[ log \Big( p(C_k) \Big) + log \Big( \sum^N_{j = 1} p_{kj}(x_i) \Big) \Big]$$

The multinomial classifier can be simplified for the Bernoulli or binary case as:

$$log \Big( p(x | C_k) \Big) = log \Big( \sum^N_{j = 1} p_{kj}^{x_i} \big( 1 -  p_{kj}^{(1 - x_i)} \big) \Big)$$


### Document classification with naive Bayes

Document classification has been one of the most sucessful applications of the naive Bayes mdoel. There is a good chance that the SPAM filter your email serivce uses is a naive Bayes model, at least in part. 


We say that we classify documents by **topics**. The naive Bayes topic model computes the probability that a document $D$ has topic $C$ based on the occurance of the words $\{ w_1, w_2, \ldots, w_n \}$, using the following relationship:

$$p(C|D) \propto \prod_{j = 1}^N p(w_j|C)$$

Notice that this topic model allows a document to have a number of topics. For example, we can say the topics of $D$ are the 5 topics with the highest probability.

For a SPAM classifier, we only need a Bernoulli topic model:

$$p(S+|D) \propto p(S+) \prod_{j=1}^N p(w_j|S+)$$

An hypothesis test is applied to each message to determine if it is SPAM. We use the log likihood ratio to determine if a given message is SPAM or not. If the following ratio is $> 1$ we classify the message as SPAM:

$$ln \Bigg( \frac{p(S+)}{p(S-)} \Bigg) = \sum_{j = 1}^N \Big[ log \big( p(w_j|S+) \big) - log \big( p(w_j|S-) \big) \Big]$$

In [ ]:
import pandas
import numpy
import seaborn
from matplotlib import pyplot

%matplotlib inline

## Example of Naive Bayes Model

Let's try a simple example of a naive bayes model. The R mlbench package contains the `HouseVotes84` which contains political party and votes on 16 important bills for 435 members of the US House of Representative. We will use this data set to build and test a classifier to predict the political party of the Congresspeople. 

Execute the code in the cell below to load the data and examine the dimensions and head of the data. 

In [ ]:
#https://archive.ics.uci.edu/ml/datasets/congressional+voting+records
votes = pandas.read_csv('house-votes-84.csv', header=None, 
                        names=['class', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7', 'V8', 'V9', 
                                   'V10', 'V11', 'V12', 'V13', 'V14', 'V15', 'V16'])
print(votes.shape)
votes.head()

each of the vote columns in the above data frame corresponds to the following topics, so let's replace the column names with something easier to remember

In [ ]:
vote_names = [
    'handicapped-infants',
    'water-project-cost-sharing',
    'adoption-of-the-budget-resolution',
    'physician-fee-freeze',
    'el-salvador-aid',
    'religious-groups-in-schools',
    'anti-satellite-test-ban',
    'aid-to-nicaraguan-contras',
    'mx-missile',
    'immigration',
    'synfuels-corporation-cutback',
    'education-spending',
    'superfund-right-to-sue',
    'crime',
    'duty-free-exports',
    'export-administration-act-south-africa']

votes.columns = ['class'] + vote_names
votes.head()

To further understand this data, let's make some plots of the first 5 votes. The code in the cell below creates bar plots for these votes by `Class` or political parties. Execute this code and examine the results.

In [ ]:
# Let's also convert our dataframe columns to "category" type to take advantage
# categorical utilities like facet-based plotting in seaborn
votes['class'] = votes['class'].astype('object').astype(
    'category', categories=['republican', 'democrat'])

values = ['y', 'n', '?']
for c in votes.columns[1:]:
    votes[c] = votes[c].astype('object').astype(
        'category', categories=values)
    
for vote_col in votes.columns[1:6]:
    seaborn.countplot(votes[vote_col])
    pyplot.show()

When examining these charts, keep in mind that there are more Democrats than Republican. What is important is the probability of Yes vote or No vote for the members of each party. Some votes are quite skewed by party affiliation, such as 'adoption of the budget resolution'. Whereas, some votes have similar probabilities by party, such as 'water project cost sharing'. These probabilities of votes by party are used to train the naive Bayes model.

Now that we understand a bit about the characteristics of the data, its time to train and test a naive Bayes model. The python `sklearn.naive_bayes` package provides a library that trains a naive Bayes model and produces a model object that can make predictions on new instances.

In [ ]:
label_col = 'class'

# We need to convert our categorical values to numeric feature vectors
feature_vecs = numpy.array([
        votes[c].cat.codes 
        for c in votes.columns 
        if c != label_col]).T
feature_vecs.shape

In [ ]:
# we also need to convert our label (democrat vs republican) to numeric values
labels = votes[label_col].cat.codes
# take a look at the mapping for the first 5 values like so
list(zip(votes[label_col][:5], labels[:5]))

In [ ]:
import sklearn.naive_bayes

# Define the model
model = sklearn.naive_bayes.MultinomialNB(alpha=1e-7)
# Train the model with our votes dataset
model.fit(feature_vecs, labels)

With the model computed, let's evaluate the performance. We can get a quick overview of the model's effectiveness by printing the first 10 rows of the result. Execute the code in the cell below to print the first 10 rows of the result and examine the result.

In [ ]:
predicted_party = model.predict(feature_vecs[:10])
party_probabilities = model.predict_proba(feature_vecs[:10])

results = pandas.DataFrame({
        'party': votes['class'][:10],
        'predicted': pandas.Categorical.from_codes(
            predicted_party, votes['class'][:10].cat.categories),
        'proba(democrat)': party_probabilities[:, 0],
        'proba(republican)': party_probabilities[:, 1],
    })
results

As you examine these results noticing:
1. There is 1 classification error, with 9 cases correctly classified.
2. In most cases, the probability of the class predicted (score) is much larger than for the other class, including for the cases with classification errors.
3. One case has nearly identical probabilities for the classes.

As a next step, we can compute the confusion matrix and performance metrics for this model. 

In [ ]:
import sklearn.metrics

def performance(results):
    accuracy = sklearn.metrics.accuracy_score(
        results['party'].cat.codes, results['predicted'].cat.codes)
    precision = sklearn.metrics.precision_score(
            results['party'].cat.codes, results['predicted'].cat.codes)
    recall = sklearn.metrics.recall_score(
            results['party'].cat.codes, results['predicted'].cat.codes)

    print('Accuracy = %.3f, Precision = %.3f, Recall = %.3f' % (accuracy, precision, recall))
    
    confusion = pandas.DataFrame(
        sklearn.metrics.confusion_matrix(
            results['party'], results['predicted']),
        index=[results.party.cat.categories], 
        columns=results.party.cat.categories)
    
    return confusion    

In [ ]:
performance(results)

The results are reasonably good looking at only the first 10 of 435 Congresspeople. 

Execute the code in the cell below computes and prints an evaluation of the model using all the data and compare the results to the first model.

In [ ]:
predicted_party = model.predict(feature_vecs)
party_probabilities = model.predict_proba(feature_vecs)

results_all = pandas.DataFrame({
        'party': votes['class'],
        'predicted': pandas.Categorical.from_codes(
            predicted_party, votes['class'].cat.categories),
        'proba(democrat)': party_probabilities[:, 0],
        'proba(republican)': party_probabilities[:, 1],
    })
performance(results_all)

The Laplace smoothing method is an effective way to deal with data sets which do not have sufficient samples to compute probabilities. This method avoids the case where $p(x_j|C_k) = 0$. 

The code in the cell below computes a naive Bayes model using the same congressional vote data, but with a Laplace smoother with a span of 3 data points. Execute this code and examine the results. 

In [ ]:
# The `alpha` param controls the Laplace smoothing
model = sklearn.naive_bayes.MultinomialNB(alpha=3)
model.fit(feature_vecs, labels)

In [ ]:
predicted_party = model.predict(feature_vecs)
party_probabilities = model.predict_proba(feature_vecs)

results_all = pandas.DataFrame({
        'party': votes['class'],
        'predicted': pandas.Categorical.from_codes(
            predicted_party, votes['class'].cat.categories),
        'proba(democrat)': party_probabilities[:, 0],
        'proba(republican)': party_probabilities[:, 1],
    })
performance(results_all)

The results are similar to the model computed without Laplace smoothing. This result is expected as all the cases in the data set have sufficient data. 

## Another Example

Let's try another binary classification example. The code in the cell below loads some sample US Census data. We want to build and evaluate a naive Bayes model to classify people by high and low income using $50,000 as the cut-off. Execute this code and examine the features in the data set. 

In [ ]:
income = pandas.read_csv('Adult Census Income Binary Classification dataset.csv', sep=', ', engine='python')
income.head()

We can see some features which are likely to be colinear. There is also one feature, 'fnlwgt', which is not useful in classifing these people. The code in the cell below removes these columns. Execute this code to create a data set with reduced features

In [ ]:
income = income.drop(['workclass', 'fnlwgt', 'education-num', 'relationship'], axis=1)
income.head()

***
**Your turn.** Compute a naive Bayes model to classify `income` using the features in the Income data set. Use `laplace = 3` for smoothing.  Print the model and examine the conditional probabilities for the values of the features to get an idea how the classifer works. 
***

In [ ]:
for c in income.columns:
    if income[c].dtype == 'object':
        income[c] = income[c].astype('category')

label_col = 'income'
labels = income.income.cat.codes

features = []
for c in income.columns:
    if c != label_col:
        if str(income[c].dtype) == 'category':
            features.append(income[c].cat.codes)
        else:
            features.append(income[c])
feature_vecs = numpy.array(features).T

model = sklearn.naive_bayes.MultinomialNB(alpha=3)
model.fit(feature_vecs, labels)

In [ ]:
conditional_probas = pandas.DataFrame(model.feature_log_prob_, columns=income.columns[:-1])
conditional_probas

In [ ]:
(-conditional_probas).plot(kind='bar')

***
**Your turn.** Using the model you computed for predicting peoples' income class, compute predictions (scores). Also compute and print the performance metrics. 
***

In [ ]:
predicted_income = model.predict(feature_vecs)
income_probabilities = model.predict_proba(feature_vecs)

accuracy = sklearn.metrics.accuracy_score(labels, predicted_income)
precision = sklearn.metrics.precision_score(labels, predicted_income)
recall = sklearn.metrics.recall_score(labels, predicted_income)
print('Accuracy = %.3f, Precision = %.3f, Recall = %.3f' % (accuracy, precision, recall))


## Changing Number of Features

Let's investigate the effect of adding more data samples to the naive Bayes model. The code in the cell below computes and evaluates naive Bayes models using 2, 3, 4, 5 and 6 votes. Execute this code and compare the results to those obtained using the full data set.

In [ ]:
for num_features in [2, 3, 4, 5, 6]:
    model = sklearn.naive_bayes.MultinomialNB()
    model.fit(feature_vecs[:, :num_features], labels)

    predicted_income = model.predict(feature_vecs[:, :num_features])
    income_probabilities = model.predict_proba(feature_vecs[:, :num_features])

    print('Number of features = %d' % num_features)
    accuracy = sklearn.metrics.accuracy_score(labels, predicted_income)
    precision = sklearn.metrics.precision_score(labels, predicted_income)
    recall = sklearn.metrics.recall_score(labels, predicted_income)
    print('Accuracy = %.3f, Precision = %.3f, Recall = %.3f' % (accuracy, precision, recall))

You can see from these results, that the model gains accuracy rapidly with just a few features. In fact, 5 or 6 of the 16 features provides equivalent results. 

***
**Your turn.** Compute a naive Bayes model, compute and  print the performance statistics for 100, 500, 1000, 2000, 8000, and 32561 rows of the Income data set. How many rows are required until the performance is close to the best possible with the model. 
***

In [ ]:
for num_rows in (100, 500, 1000, 2000, 8000, 32561):
    model = sklearn.naive_bayes.MultinomialNB()
    model.fit(feature_vecs[:num_rows, :], labels[:num_rows])

    predicted_income = model.predict(feature_vecs[:num_rows, :])
    income_probabilities = model.predict_proba(feature_vecs[:num_rows, :])
    
    accuracy = sklearn.metrics.accuracy_score(labels[:num_rows], predicted_income)
    precision = sklearn.metrics.precision_score(labels[:num_rows], predicted_income)
    recall = sklearn.metrics.recall_score(labels[:num_rows], predicted_income)
    print('Accuracy = %.3f, Precision = %.3f, Recall = %.3f' % (accuracy, precision, recall))

## Summary

In this notebook you have worked through the following:

1. Theory of naive Bayes models.
2. Examples of computing and evaluating naive Bayes models.
3. Examine the effects of data set size on the results of naive Bayes models. 

#### Copyright 2017, Stephen F Elston. All rights reserved. 